In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs

In [ ]:
%%time
a,l = load_rfam(full= True, add_cov= False)

In [ ]:
%%time
def atograph(ali):
    graph = ali2graph.writecons(ali)
    graph = ali2graph.dillute( graph, dilute1 = .7, dilute2 = .4 , fix_edges =False)
    graph = ali2graph.conscut( graph,  consThresh= .93, replacelabel = False )
    graphs = ali2graph.nearstem( graph, boost_range = 4, boost_thresh = .5, boost_weight = 1)
    return graph

graphs = ut.xxmap(atograph, a)
matrix = ygraphs.vectorize_graphs(graphs, normalization=True,
                      min_r = 2,min_d=1,
                      inner_normalization=True)

# sml.knn_accuracy(matrix,l)

In [ ]:
%%time
from  sklearn.neighbors import KNeighborsClassifier
def calcranks(matrix,l):
    knn = KNeighborsClassifier(n_neighbors=500+1)
    yy= np.array(l)
    knn.fit(matrix,yy)
    _, indices = knn.kneighbors(matrix)
    neighbor_labels = yy[indices]
    neighbor_labels = neighbor_labels[:, 1:]
    def searchrank(i,l):
        row = neighbor_labels[i] 
        indices = np.where(row == l)[0]
        return indices[0] if len(indices)>0 else 999999
    ranks = [searchrank(i,lab) for i,lab in enumerate(yy) if lab != 0]
    return np.array(ranks)
ranks = calcranks(matrix, l)

In [ ]:
%%time
a2,l2 = load_rfam(full= True, add_cov= False)
mat = ygraphs.vectorize_alignments(a, min_r = 2, min_d=1)
ranks2 = calcranks(mat,l2)

In [ ]:
# PLOT importance weighted vectorization  vs normal vectorization , do we have at least 1 NN in the closest X instances?
plt.plot([sum(ranks < x)/sum(l != 0) for x in range(1,11)])
plt.plot([sum(ranks2 < x)/sum(l2 != 0) for x in range(1,11)])

In [ ]:
plt.plot([sum(ranks < x)/sum(l != 0) for x in range(500)])
plt.plot([sum(ranks2 < x)/sum(l2 != 0) for x in range(500)])

In [ ]:
# are there different successrates for more populated families? 
manyseq, fewseq = filter_by_seqcount(a)

def pl(x):
    x.append(0)
    mask = [ ll in x for ll in l ]
    ranks = calcranks(matrix[mask], l[mask])
    # ranks = calcranks(mat[mask], l[mask])
    data = [sum(ranks < xx)/sum(l[mask] != 0) for xx in range(1,21)]
    plt.plot(data)
    return data

dg = pl(manyseq)
db = pl(fewseq)

In [ ]:
todi = lambda x,y: [{'clan hit':e, 'neighbors':i, 'experiment':y} for i,e in enumerate(x)]
import pandas as pd
import seaborn as sns
g,b = ([0.26717557251908397,
  0.2824427480916031,
  0.29770992366412213,
  0.32061068702290074,
  0.33587786259541985,
  0.3511450381679389,
  0.35877862595419846,
  0.3816793893129771,
  0.3816793893129771,
  0.3969465648854962,
  0.40458015267175573,
  0.40458015267175573,
  0.4122137404580153,
  0.4122137404580153,
  0.4122137404580153,
  0.4122137404580153,
  0.4198473282442748,
  0.42748091603053434,
  0.4351145038167939,
  0.45038167938931295],
 [0.3231939163498099,
  0.3726235741444867,
  0.41064638783269963,
  0.43346007604562736,
  0.44106463878326996,
  0.4600760456273764,
  0.4714828897338403,
  0.4790874524714829,
  0.49049429657794674,
  0.49429657794676807,
  0.5057034220532319,
  0.5171102661596958,
  0.5209125475285171,
  0.5285171102661597,
  0.5285171102661597,
  0.5285171102661597,
  0.5285171102661597,
  0.532319391634981,
  0.532319391634981,
  0.5361216730038023])
data = pd.DataFrame(todi(dg,'manySequences, boost_flank')+
                    todi(db,'fewSequences, boost_flank')+
                    todi(g,'manyseq, no boost')+
                    todi(b,'fewseq, no boost'))
# sns.lineplot(x="neighbors", y="clan hit", hue="experiment",  data=data)

data = pd.DataFrame(todi(dg,'manySequences')+
                    todi(db,'fewSequences'))

sns.lineplot(x="neighbors", y="clan hit",
             hue="experiment", 
             data=data)

In [ ]:
ranks = calcranks(matrix, l)

In [ ]:
ranks

In [ ]:
graphs[0].nodes[2]

In [ ]:
# if we know the neighbor, can we find the whole familiy? 
knn = KNeighborsClassifier(n_neighbors=100)
yy= np.array(l)
knn.fit(matrix,yy)
_, indices = knn.kneighbors(matrix[yy!=0])

interesting = indices[ranks == 0]

# are two enough to find the rest? 

In [ ]:
vec = eg.Vectorizer( min_r = 2,min_d=1)

# project other instances to our thing.. 

# 

for line in interesting:
    c,d = line[0], line[1]
    needle = np.array([a * b for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0])])

    input = np.log1p(matrix[[c,d],:])
    needle = np.asarray(np.sum(input, axis = 0))[0]
    
    input = matrix[[c,d],:]
    needle = np.asarray(np.sum(input, axis = 0))[0]
    
    dist, indices = knn.kneighbors([needle])
    print(yy[indices])
    print(f'{sum(yy == yy[indices][0,0])=}')
    print(dist)
    
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle
        return graph
    gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 

    def draw(g):
        if len(g) < 400: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    [draw(g) for g in gra]



In [ ]:
# CAN WE FIND NEW INSTANCES OF A CLUSTER IF WE PROVIDE THE WHOLE CLUSTER?

knn = KNeighborsClassifier(n_neighbors=100)
yy= np.array(l)
knn.fit(matrix ,yy)

y_needle =  {}

for y in np.unique(yy): 
    if y > 0:
        
        input = np.log1p(matrix[yy == y])
        needle = np.asarray(np.sum(input, axis = 0))
        
        # input = matrix[yy == y]
        # needle = np.asarray(np.all((input > 0).todense(), axis = 0))
        
        dist, indices = knn.kneighbors(needle)
        print(y)
        print(yy[indices])
        print(dist)
        y_needle[y] = (needle, indices) 


In [ ]:
def show(y):
    needle, indices = y_needle[y]
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle[0]
        return graph
    gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    
    def draw(g):
        if len(g) < 300: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    [draw(g) for g in gra]


show(8)
